### IMPORTS

In [1]:
import ipynb.fs.full.processing as processing
import ipynb.fs.full.training as training
import ipynb.fs.full.storage as storage
import ipynb.fs.full.misc as misc
import ipynb.fs.full.splitting as splitting
import ipynb.fs.full.features as features
import ipynb.fs.full.ensemble as ensemble

In [2]:
import json

### SERIALIZE THE YAML CONFIG

In [3]:
config = storage.load_yaml('extra/config.yaml')

In [4]:
config

{'data': 'extra/dataset.csv',
 'processing': {'resample': True,
  'time': 'D',
  'aggregate': {'Open': 'first',
   'High': 'max',
   'Low': 'min',
   'Close': 'last',
   'Volume': 'sum'},
  'label': {'from': 'Close', 'shift': 1}},
 'features': {'category': 'all', 'window': 14},
 'splitting': {'train_split': 0.8, 'validation_folds': 5},
 'regression_ensemble': {'models': [{'linreg': None},
   {'lstm': {'morph': {'window': 4, 'batch': 30},
     'layers': [{'lstm': {'units': 120}},
      {'dropout': {'rate': 0.15}},
      {'dense': {'units': 50, 'activation': 'relu'}},
      {'dense': {'units': 1}}],
     'epochs': 15,
     'loss': 'mean_squared_error',
     'optimizer': 'rmsprop'}},
   {'tcn': {'morph': {'window': 4, 'batch': 30},
     'layers': [{'tcn': {'nb_filters': 64,
        'nb_stacks': 1,
        'dilations': [1, 2, 4, 8, 16, 32],
        'padding': 'causal',
        'use_skip_connections': False,
        'dropout_rate': 0.1,
        'return_sequences': False}},
      {'dropout':

### STEP 1: PROCESS RAW DATA TO DATAFRAME

In [5]:
dataframe = processing.create_dataframe(config)

In [6]:
dataframe.head(5)

,Open,High,Low,Close,Volume,label
Date_Timestamp,,,,,,
2010-01-01,1.43327,1.43356,1.43207,1.43335,39761.000053,1.43141
2010-01-03,1.43024,1.43359,1.42951,1.43141,3001.600003,1.44244
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.400094,1.43634
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.100067,1.44005
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.800085,1.43155


### STEP 2: ADD FEATURES TO DATAFRAME

In [7]:
regression_dataset = features.add(dataframe, config['features'])

In [8]:
regression_dataset.head(5)

,open,high,low,close,volume,label,stock,stocd,stocsd,momentum,...,rsi,obv,ma,bias6,psy12,asy5,asy4,asy3,asy2,asy1
Date_Timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-21,1.41115,1.41422,1.40247,1.40979,78980.100041,1.41361,13.189189,9.578919,19.849579,-0.02655,...,29.873956,-159980.600046,1.425180,-0.012336,41.7,-0.403064,-0.471831,-0.715772,-0.637503,-0.099256
2010-01-22,1.40977,1.41805,1.40837,1.41361,76660.300037,1.41517,20.072072,13.339429,14.445538,-0.02644,...,33.983332,-83320.300009,1.420572,-0.006774,41.7,-0.323346,-0.469180,-0.334804,0.085670,0.270596
2010-01-24,1.41557,1.41705,1.41396,1.41517,3128.600004,1.41458,22.882883,18.714715,13.877688,-0.01638,...,35.641946,-80191.700005,1.415528,-0.003171,33.3,-0.353285,-0.223529,0.093878,0.190445,0.110295
2010-01-25,1.41498,1.41939,1.41248,1.41458,79577.200048,1.40808,21.819820,21.591592,17.881912,-0.02625,...,35.280916,-159768.900053,1.412868,-0.000558,33.3,-0.187163,0.059984,0.113064,0.034298,-0.041700
2010-01-26,1.41454,1.41780,1.40397,1.40808,79232.600053,1.40280,10.108108,18.270270,19.525526,-0.03823,...,31.495786,-239001.500106,1.412246,-0.002826,25.0,-0.044125,-0.030342,-0.130655,-0.251130,-0.460559


### STEP 3: SPLIT PRIMARY DATA INTO TRAIN & TEST SECTIONS

In [9]:
primary_dataset = splitting.general(
    regression_dataset,
    config['splitting']['train_split']
)

### STEP 4: CREATE CROSS VALIDATION FOLDS & TRAIN REGRESSION MODELS

In [10]:
regression_ensemble, regression_table = ensemble.regression(primary_dataset, config)

TRAINING LINREG_0 FOLD #1
TRAINING LINREG_0 FOLD #2
TRAINING LINREG_0 FOLD #3
TRAINING LINREG_0 FOLD #4
TRAINING LINREG_0 FOLD #5

TRAINING LSTM_1 FOLD #1
Epoch 1/15
14/14 [==============================] - 0s 22ms/step - loss: 0.2953 - mean_squared_error: 0.2953
Epoch 2/15
14/14 [==============================] - 0s 22ms/step - loss: 0.0313 - mean_squared_error: 0.0313
Epoch 3/15
14/14 [==============================] - 0s 22ms/step - loss: 0.0265 - mean_squared_error: 0.0265
Epoch 4/15
14/14 [==============================] - 0s 23ms/step - loss: 0.0223 - mean_squared_error: 0.0223
Epoch 5/15
14/14 [==============================] - 0s 23ms/step - loss: 0.0123 - mean_squared_error: 0.0123
Epoch 6/15
14/14 [==============================] - 0s 22ms/step - loss: 0.0207 - mean_squared_error: 0.0207
Epoch 7/15
14/14 [==============================] - 0s 23ms/step - loss: 0.0145 - mean_squared_error: 0.0145
Epoch 8/15
14/14 [==============================] - 0s 23ms/step - loss: 0.0142 - 

69/69 [==============================] - 2s 24ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 14/15
69/69 [==============================] - 2s 24ms/step - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 15/15
69/69 [==============================] - 2s 24ms/step - loss: 0.0040 - mean_squared_error: 0.0040

TRAINING TCN_2 FOLD #1
Epoch 1/15
14/14 [==============================] - 0s 4ms/step - loss: 6.1714 - mean_squared_error: 6.1714
Epoch 2/15
14/14 [==============================] - 0s 5ms/step - loss: 0.9336 - mean_squared_error: 0.9336
Epoch 3/15
14/14 [==============================] - 0s 5ms/step - loss: 0.4145 - mean_squared_error: 0.4145
Epoch 4/15
14/14 [==============================] - 0s 5ms/step - loss: 0.2440 - mean_squared_error: 0.2440
Epoch 5/15
14/14 [==============================] - 0s 4ms/step - loss: 0.2668 - mean_squared_error: 0.2668
Epoch 6/15
14/14 [==============================] - 0s 4ms/step - loss: 0.1840 - mean_squared_error: 0.1840
Epoch 7/15

69/69 [==============================] - 0s 5ms/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 13/15
69/69 [==============================] - 0s 4ms/step - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 14/15
69/69 [==============================] - 0s 5ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 15/15
69/69 [==============================] - 0s 5ms/step - loss: 0.0043 - mean_squared_error: 0.0043


In [11]:
regression_table.head(5)

,linreg_0,lstm_1,tcn_2,label
0,1.415414,1.290080,1.182397,1.40686
1,1.411078,1.285593,1.195333,1.41436
2,1.419328,1.285042,1.260293,1.43163
3,1.433976,1.294058,1.396100,1.42894
4,1.422776,1.288351,1.548195,1.43328


### STEP 6: CREATE DECISION MACHINE & PUT LABELS THROUGH IT

In [12]:
decision_machine = misc.decision_machine()

In [13]:
regression_labels = decision_machine.calibrate(
    regression_table,
    config['classification_ensemble']['decision']
)

In [14]:
labeled_regression_table = misc.replace_labels(
    regression_table,
    regression_labels
)

In [15]:
# BUY   = 0
# SELL  = 1
# HOLD  = 2

In [16]:
labeled_regression_table.head(5)

,linreg_0,lstm_1,tcn_2,label
0,1.415414,1.290080,1.182397,2
1,1.411078,1.285593,1.195333,1
2,1.419328,1.285042,1.260293,1
3,1.433976,1.294058,1.396100,2
4,1.422776,1.288351,1.548195,1


### DECISION BREAKDOWN

In [17]:
labeled_regression_table['label'].value_counts(normalize=True)

2    0.500483
1    0.249758
0    0.249758
Name: label, dtype: float64

In [18]:
storage.save_pickle(labeled_regression_table, 'extra/cls-train.pickle')

### STEP 7: TRAIN THE CLASSIFIER ENSEMBLE

In [19]:
classifier_ensemble = ensemble.classifier(
    labeled_regression_table,
    config
)

TRAINING RANDFOREST_0 MODEL
TRAINING LOGREG_1 MODEL
TRAINING SVC_2 MODEL


### STEP 8: REGRESSION PREDICT ON TEST DATASET

In [20]:
test_predictions = regression_ensemble.predict(primary_dataset['test'])

In [21]:
test_predictions.head(5)

,linreg_0,lstm_1,tcn_2
0,1.196712,1.120126,1.148302
1,1.200233,1.117770,1.144689
2,1.208944,1.121186,1.161010
3,1.220007,1.129053,1.131946
4,1.218342,1.147335,1.203475


### STEP 10: ADD LABELS TO TEST PREDICTIONS

In [25]:
test_results = test_predictions.copy()

In [26]:
test_results['label'] = primary_dataset['test']['labels'][-len(test_results):]

In [39]:
test_results.head(5)

,linreg_0,lstm_1,tcn_2,label
0,1.196712,1.120126,1.148302,1.19558
1,1.200233,1.117770,1.144689,1.20427
2,1.208944,1.121186,1.161010,1.21993
3,1.220007,1.129053,1.131946,1.21919
4,1.218342,1.147335,1.203475,1.22665


### STEP 9: CLASSIFY PREDICT ON REGRESSION DATASET

In [22]:
mash_dataset = {
    'features': test_predictions.to_numpy(),
    'labels': []
}

In [23]:
cls_dataset = classifier_ensemble.predict(mash_dataset)

In [24]:
cls_dataset.head(5)

,randforest_0,logreg_1,svc_2
0,0,1,2
1,0,1,2
2,0,1,2
3,0,1,2
4,2,0,2


### STEP 10: ADD CLASSIFIED LABELS TO IT

In [37]:
classified_labels = decision_machine.convert(test_results)

In [41]:
cls_dataset['label'] = classified_labels

In [48]:
cls_dataset.head(5)

,randforest_0,logreg_1,svc_2,label
0,0,1,2,2
1,0,1,2,1
2,0,1,2,1
3,0,1,2,2
4,2,0,2,1


### CONFUSION MATRIX TESTING

In [67]:
from sklearn.metrics import confusion_matrix

In [68]:
second = cls_dataset['logreg_1'].to_numpy()

In [69]:
conf_matrix = confusion_matrix(first, second)

In [70]:
conf_matrix

array([[ 55,  61,   3],
       [ 48,  58,   8],
       [198, 174,  13]], dtype=int64)

### STEP 11: STITCH TOGETHER REGRESSION FITTINGS

In [27]:
regression_fitting = {}

In [28]:
for blob in regression_ensemble.models:
    collection = {}
    for index, model in enumerate(blob):
        collection['fold_' + str(index)] = model.score
        
    regression_fitting[model.name] = collection

### STEP 12: STITCH TOGETHER CLASSIFIER FITTINGS

In [29]:
classifier_fitting = {}

In [30]:
for model in classifier_ensemble.models:
    classifier_fitting[model.name] = model.score

### STEP X: SAVE EVERYTHING

In [31]:
storage.save_pipeline({
    'config': config,
    'regression_ensemble': regression_ensemble,
    'classifier_ensemble': classifier_ensemble,
    'predictions': {
        'training': json.loads(regression_table.to_json()),
        'validation': json.loads(test_results.to_json())
    },
    'fittings': {
        'regression': regression_fitting,
        'classifier': classifier_fitting
    }
})